# Team Purple - Group project
**Team members**: Jayant Maheshwari, Shubham Rishishwar, Sharath Reddy, Rajat Gaur

# Importing relevant libraries

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import nltk
import keras
import spacy
import functools
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.ensemble import GradientBoostingClassifier
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from random import randint
from numpy import array, argmax, asarray, zeros
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
from sklearn import linear_model
warnings.filterwarnings("ignore")
nlp = spacy.load("en_core_web_md") 

# 0. Read inputs

In [3]:
# Reading 4 Data Frames from "Project Code - Part 1.ipynb" (used for data preprocessing)

df_color = pd.read_csv("Primary color.csv",index_col = 0)
df_fit = pd.read_csv("fit.csv",index_col = 0)
df_occasion = pd.read_csv("occasion.csv",index_col = 0)
df_style = pd.read_csv("style.csv", index_col = 0)

In [4]:
# transforming T/F to binary (our labels start from 6th column, so we use slicing from [6:])
# We convert these labels from TRUE/FALSE to 1/0)

df_color.iloc[:,6:]= df_color.iloc[:,6:].astype(int)
df_fit.iloc[:,6:]= df_fit.iloc[:,6:].astype(int)
df_occasion.iloc[:,6:]= df_occasion.iloc[:,6:].astype(int)
df_style.iloc[:,6:]= df_style.iloc[:,6:].astype(int)

In [5]:
df_color.head(2)            # sample view of dataframe

,product_id,brand,product_full_name,description,brand_category,details,blacks,pinks,whites,reds,...,grays,golds,navy,yellows,burgundies,purples,browns,multi,oranges,teal
0,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,Designed worn high-waisted bottom oh-so-now mo...,Unknown,Designed worn high-waisted bottom oh-so-now mo...,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,01DVA59VHYAPT4PVX32NXW91G5,Tibi,Juan Embossed Mules,Tibis Juan embossed mule made shiny black leat...,women:SHOES:MULES,seen Pre-Fall ‘19 runway Heel measure approxim...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# combine multiple (relevant) columns into a single document

doc_color = df_color.brand + df_color.product_full_name + df_color.description + df_color.brand_category + df_color.details
doc_fit = df_fit.brand + df_fit .product_full_name + df_fit.description + df_fit.brand_category + df_fit.details
doc_occasion = df_occasion.brand + df_occasion.product_full_name + df_occasion.description + df_occasion.brand_category + df_occasion.details
doc_style = df_style.brand + df_style.product_full_name + df_style.description + df_style.brand_category + df_style.details

# 1. Count Vectorizer with Logistic Regression model

#### 42 models trained, one for each of the 42 labels, across our 4 categories, viz., 'Occasion', 'Style', 'Fit', and 'Color'

> Accuracy across the models, on average, was consistently higher than the baseline accuracy.

In [7]:
## 1. We first Count vectorize the above document by fit_transform method
## 2. Next, we standardize the numbres to bring them to same scale
## 3. We split the data into train: test (75:25)
## 4. Then, we trained the above mentioned 42 models on our data, and stored them in a list for further use

def logistic_model(doc,df,columns):
    vectorizer = CountVectorizer()
    vectorizer = vectorizer.fit(doc)
    X = vectorizer.transform(doc)                                            # count vectorize the input
    X = X.toarray()
    
    scaler = StandardScaler()                 
    scaler = scaler.fit(X)
    X = scaler.transform(X)                                                  # standardize data
    data = pd.DataFrame(X, columns=vectorizer.get_feature_names())           # Create a dataframe
    
    models = []
    
    for col in columns:
        y = df[col].values
        #base_accuracy = y.sum()/len(y)
        #base_accuracy = max(base_accuracy,1-base_accuracy)

        data["TARGET"] = y

        train_df, test_df = train_test_split(data)                           # Train-test split
        X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
        X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

        y_train = train_df["TARGET"]
        y_test = test_df["TARGET"]

        # Logistic model
        clf =linear_model.LogisticRegression(C=0.001,random_state=None).fit(X_train, y_train)

        # Predict labels
        #y_pred = clf.predict(X_test)
        
        # check accuracy
        #acc = np.mean(y_pred == y_test)
        
        models.append(clf)
    return models, vectorizer, scaler

In [8]:
# These are the 42 labels for which models will be fitted

columns_color = ['blacks','pinks','whites','reds','greens','blues','silvers','neutrals','oranges',
                'beiges','grays','golds','navy','yellows','burgundies','purples','browns','multi','teal']
columns_fit   = ['business casual','classic','modern','boho','glam','romantic','casual','androgynous','edgy',
                 'retro','athleisure']
columns_occasion = ['day to night','work','weekend','night out','vacation','coldweather','workout'] 
columns_style = ['semi-fitted','relaxed','straight / regular','fitted / tailored','oversized']

columns_list = columns_color + columns_fit + columns_occasion + columns_style

# call the logistic regeression function for each category
model_color, vector_color, scaler_color = logistic_model(doc_color,df_color,columns_color)
model_fit, vector_fit, scaler_fit = logistic_model(doc_fit,df_fit,columns_fit)
model_occasion, vector_occasion, scaler_occasion = logistic_model(doc_occasion,df_occasion,columns_occasion)
model_style, vector_style, scaler_style = logistic_model(doc_style,df_style,columns_style)

# combine the models into a single list
model_list = model_color+model_fit+model_occasion+model_style

# 2. Test on new data

In [9]:
# test data to be input here (this is dummy data):
test_brand = "Forever 21"
test_product_full_name = "Jeans size 34 M,"
test_description = "This is a slim jeans"
test_brand_category = "Denim Jeans"
test_details = "Blue color"

test_docs = test_brand +" " + test_product_full_name + " " + test_description + " " + test_brand_category +\
            " " + test_details

# Remove Punctuations
punctuation = "!@#$%^&*()_+<>?:.,;"  
    
for c in test_docs:
    if c in punctuation:
        test_docs = test_docs.replace(c, "")

        
# Remove Stopwords  
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(test_docs) 
test_docs = [w for w in word_tokens if not w in stop_words] 
test_docs = [] 
for w in word_tokens: 
    if w not in stop_words: 
        test_docs.append(w)

        
# Combine all cleaned words into a list
test_docs = [' '.join(test_docs)]

In [10]:
# Vectorize and standardize the test documnets using the models fit in Step 1.

Xtest_color = vector_color.transform(test_docs)
Xtest_color = Xtest_color.toarray()
Xtest_color = scaler_color.transform(Xtest_color)
Xtest_color = pd.DataFrame(Xtest_color, columns=vector_color.get_feature_names())

Xtest_fit = vector_fit.transform(test_docs)
Xtest_fit = Xtest_fit.toarray()
Xtest_fit = scaler_fit.transform(Xtest_fit)
Xtest_fit = pd.DataFrame(Xtest_fit, columns=vector_fit.get_feature_names())

Xtest_occasion = vector_occasion.transform(test_docs)
Xtest_occasion = Xtest_occasion.toarray()
Xtest_occasion = scaler_occasion.transform(Xtest_occasion)
Xtest_occasion = pd.DataFrame(Xtest_occasion, columns=vector_occasion.get_feature_names())

Xtest_style = vector_style.transform(test_docs)
Xtest_style = Xtest_style.toarray()
Xtest_style = scaler_style.transform(Xtest_style)
Xtest_style = pd.DataFrame(Xtest_style, columns=vector_style.get_feature_names())

In [11]:
# Combine all labels for which [Prediction = 1] on test_data into a single dataframe

y_pred = []
for mod in model_list[:19]:                            # first 19 labels are for Category: COLOR
    pred = list(mod.predict(Xtest_color))
    y_pred.append(pred)

for mod in model_list[19:30]:                          # next 11 labels are for Category: FIT
    pred = list(mod.predict(Xtest_fit))
    y_pred.append(pred)
    
for mod in model_list[30:37]:                          # next 7 labels are for Category: OCCASION
    pred = list(mod.predict(Xtest_occasion))
    y_pred.append(pred)
    
for mod in model_list[37:42]:                          # last 5 labels are for Category: STYLE
    pred = list(mod.predict(Xtest_style))
    y_pred.append(pred)    

results = pd.DataFrame(y_pred, index = columns_list, columns = ['Predicted labels'])
results[results['Predicted labels']>0]

,Predicted labels
casual,1
day to night,1
weekend,1


# 4. Other models inspected

### Define all necessary functions for later use

In [78]:
# integere encoding for the documents
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

# call the standard Stanford-trained GLOVE vectors to get the weight matrix (embeddings)
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt', encoding = 'utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Fit the Recurrent Neural Network (RNN) model on data
def make_binary_classification_rnn_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(16))
    model.add(Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    return model

# Fit the Long Short-term Memory (LSTM) model on data
def make_lstm_classification_model(plot = False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=32, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()

    return model

## 4a. Glove + LSTM Model 

In [99]:
# Tokenize Text
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(list(doc_color))

# integer encode the documents
encoded_docs = integer_encode_documents(doc_color, tokenizer)

# padding to create equal length sequences
MAX_SEQUENCE_LENGTH = 200
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df_color['blacks']))

# train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

# Load in GloVe Vectors
embeddings_index = load_glove_vectors()
embeddings_index

# # create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector
        
# define model
model = make_lstm_classification_model()

# fit the model
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          687900    
_________________________________________________________________
masking_4 (Masking)          (None, 200, 100)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total params: 704,990
Trainable params: 17,090
Non-trainable params: 687,900
_________________________________________________________________
Train on 659 samples, validate on 74 samples
Epoch 1/5
659/659 [==============================] - ETA: 17s - loss: 0.7708 - accuracy: 0.250 - ETA: 10s - loss: 0.7218 - accuracy: 0.421 - ETA: 7s - loss: 0.7018 - accuracy: 0.500 - ETA: 6s - lo

In [105]:
print('Baseline accuracy for label Black (sample label):', 100*(1 - df_color['blacks'].sum()/len(df_color['blacks'])), '%')
print('LSTM Model accuracy for label Black (sample label):', 100*accuracy, '%')

Baseline accuracy for label Black (sample label): 67.61177753544165 %
LSTM Model accuracy for label Black (sample label): 66.847825050354 %


In [101]:
# WE DID NOT select this model, as accuracy was similar to baseline accuracy 
## Test on new data

test_docs = [
    "Employees look like they hate their job. Milkshake was like drinking milk. Food was cold and not warm at all",
    "This Mcdonalds is not only in the business of making crappy food and providing even crappier service watch out for the racket they have in the parking lot . If your not careful reading the sign at the the front of the entrance it is going to cost you $195.00 in parking fees. went in to to ask the management they just blew me off. lucky they are in vegas where they dont count on repeat businesssss.",
    "There are better stores without fruit flies in Griffin, GA.",
    "Slowest drive-thru ever. Better option is to go to the location on arlington"
]

test_docs = list(
    map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), test_docs))

encoded_test_sample = integer_encode_documents(test_docs, tokenizer)

padded_test_docs = keras.preprocessing.sequence.pad_sequences(encoded_test_sample, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

model.predict_classes(padded_test_docs)
prediction = model.predict_classes(padded_test_docs)
encoder.inverse_transform(prediction)

array([0, 0, 0, 0])

## 4b. Glove + RNN Model 

In [102]:
# Tokenize Text
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(list(doc_color))

# integer encode the documents
encoded_docs = integer_encode_documents(doc_color, tokenizer)

# padding to create equal length sequences
MAX_SEQUENCE_LENGTH = 200
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df_color['blacks']))

# train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

# Load in GloVe Vectors
embeddings_index = load_glove_vectors()
embeddings_index

# # create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector
        
# define model
model = make_binary_classification_rnn_model()

# fit the model
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          687900    
_________________________________________________________________
masking_5 (Masking)          (None, 200, 100)          0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                10560     
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 34        
Total params: 699,534
Trainable params: 11,634
Non-trainable params: 687,900
_________________________________________________________________
Train on 659 samples, validate on 74 samples
Epoch 1/5
659/659 [==============================] - ETA: 13s - 

In [104]:
print('Baseline accuracy for label Black (sample label):', 100*(1 - df_color['blacks'].sum()/len(df_color['blacks'])), '%')
print('RNN Model accuracy for label Black (sample label):', 100*accuracy, '%')

Baseline accuracy for label Black (sample label): 67.61177753544165 %
RNN Model accuracy for label Black (sample label): 66.847825050354 %


# Work in progress

### LSTM + Word2Vec (Equal Weights)

**unable to finally generate any meaningful output**

In [ ]:
# Tokenize Text
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(list(doc_color))

# integer encode the documents
encoded_docs = integer_encode_documents(doc_color, tokenizer)

# padding to create equal length sequences
MAX_SEQUENCE_LENGTH = 1000
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df_color['blacks']))

# train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

# Load in GloVe Vectors
embedding_matrix = []
for i in doc_color:
    embedding_matrix.append(nlp(i).vector)
embedding_matrix = np.asarray(embedding_matrix)

# embeddings_index = load_glove_vectors()
# embeddings_index

#create a weight matrix for words in training docs
# embedding_matrix = zeros((VOCAB_SIZE, 100))
# for word, i in tokenizer.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: # check that it is an actual word that we have embeddings for
#         embedding_matrix[i] = embedding_vector


# define model
model = make_lstm_classification_model2()

# fit the model
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
nlp(doc).vector

In [ ]:
# test = "black shoes green belt"
# nlp(test).vector

### Self Trained Corpus
Generate word vectors by self-training the weights on corpus 

In [107]:
doc = list(doc.values)
doc = [word_tokenize(review) for review in doc]
model = Word2Vec(doc, min_count=5)
words = list(model.wv.vocab)
vectors = []
for word in words:
    vectors.append(model[word].tolist())
data = np.array(vectors)
data